In [ ]:
import civis
import os
import requests
import json
import pandas as pd
import urllib.parse
from pathlib import Path
from datetime import date
from datetime import timedelta


# headers needed for GET requests & GLOBALS
GET_HEADERS = {'X-Knack-REST-API-KEY':'1a210580-315e-11ea-a6a4-bb031a9e1ba1', 'X-Knack-Application-Id':'5e13989941e72c0e039e117f'}
POST_HEADERS = GET_HEADERS.copy()
POST_HEADERS.update({'content-type': 'application/json'})

YESTERDAY_DATE = (date.today() - timedelta(days = 1)).strftime("%m/%d/%Y")
DATE_TO_RUN = YESTERDAY_DATE

# Knack entity mappings for readability
knackmappingdict = {
    'LocalDuesCategory':{'objectid':'object_37', 'ID':'field_451', 'GUID':'field_870', 'NAME':'field_452', 'UpdatedinAFTDB':'field_1055'},
    'Employer':{'objectid':'object_16', 'ID':'field_263', 'GUID':'field_854', 'NAME':'field_265', 'UpdatedinAFTDB':'field_1068'},
    'LocalAgreement':{'objectid':'object_23', 'ID':'field_291', 'GUID':'field_856', 'NAME':'field_292', 'UpdatedinAFTDB':'field_1064'},
    'Unit':{'objectid':'object_22', 'ID':'field_319', 'GUID':'field_858', 'NAME':'field_321', 'UpdatedinAFTDB':'field_1065'},
    'LocalJobClass':{'objectid':'object_21', 'ID':'field_343', 'GUID':'field_860', 'NAME':'field_345', 'UpdatedinAFTDB':'field_1066'},
    'JobTitle':{'objectid':'object_24', 'ID':'field_355', 'GUID':'field_862', 'NAME':'field_356', 'UpdatedinAFTDB':'field_1067'},
    'WorkLocation':{'objectid':'object_28', 'ID':'field_389', 'GUID':'field_864', 'NAME':'field_391', 'UpdatedinAFTDB':'field_1062'},
    'WorkStructure':{'objectid':'object_29', 'ID':'field_412', 'GUID':'field_866', 'NAME':'field_414', 'UpdatedinAFTDB':'field_1063'}
}


# Tries to get knackId of given object with match params. Returns val if found blank otherwise
def getKnackID(knack_object_id, guid_field, guid_value, name_field, name_value):
    #Get Id
    match_filter = {'match':'and', 'rules':[{'field':guid_field, 'operator':'is', 'value': guid_value},
                                           {'field':name_field, 'operator':'is', 'value': name_value}]}
    filter_for_url = urllib.parse.quote(json.dumps(match_filter))
    request_url = "https://api.knack.aft.org/v1/objects/" + knack_object_id + "/records?filters=" + filter_for_url
    r = requests.get(url = request_url, headers = GET_HEADERS)
    #print(json.dumps(r.json(), indent=4))
    res_json_dict = json.loads(json.dumps(r.json()))
    if res_json_dict["total_records"] == 1:
        return res_json_dict["records"][0]["id"]
    else:
        return ''


def upload_record_to_knack(payloads, entity):
    entity_objectid = knackmappingdict[entity]['objectid']
    
    guid_field = knackmappingdict[entity]['GUID']
    name_field = knackmappingdict[entity]['NAME']
    
    id_field = knackmappingdict[entity]['ID']
    update_field = knackmappingdict[entity]['UpdatedinAFTDB']

    count = 0
    lenght = len(payloads)
    print(f'Found {lenght} records:')

    for record in payloads:
        count += 1
        record_name = record[entity.lower()+'name']
        record_guid = record['knackfieldguid']
        record_id = record[entity.lower()+'id']
        updateval = record['processedinaftdb']

        print(f'  Record {count}/{lenght}:')
        print(f'    name={record_name}')
        print(f'    knackid={record_guid}')
        print(f'    {entity}id={record_id}')


        if record_id:
            failcheck = getKnackID(entity_objectid, guid_field, record_guid, name_field, record_name)

            if failcheck:
                print(f'  Found {entity}, return id= {failcheck}')
                request_url = "https://api.knack.aft.org/v1/objects/" + entity_objectid + "/records/" + failcheck
                r = requests.put(url = request_url, headers = POST_HEADERS, data = json.dumps({id_field:record_id, update_field:updateval}))
                if r.status_code != 200:
                    print(json.dumps(r.json(), indent=4))
                else:
                    print(f'  Updated {entity}ID in Knack to {record_id}')
                    print(f'  Successfly uploaded to knack! Response: {r}')
            else:
                print(f'  No {entity} found with KnackID:{record_guid} and name:{record_name}')
        else:
            print("  No ID provided to update record")


#Runner for localdues
def run_table(entity):
    input_schema = os.environ['table_schema']
    table_name = input_schema + entity.lower()
    print(f'Loading Table: {table_name}')
    f = civis.io.read_civis(table=table_name,
                            database="American Federation of Teachers",
                            use_pandas=True)
    f.fillna('', inplace=True)
    
    payloads = []
    for record in f.to_dict('records'):
        if record['updatedat'] == DATE_TO_RUN:
            if record['action'] == 'Insert' or record['action'] == 'Update':
                keys_to_extract = ["knackfieldguid", entity.lower()+'id', entity.lower()+'name', 'processedinaftdb']
                payload = {key: record[key] for key in keys_to_extract}
                payloads.append(payload)      
    upload_record_to_knack(payloads, entity)

def main():
    entity_list = list(knackmappingdict.keys())

    for entity in entity_list:
        print(f'========== Running Entity: {entity} ==========')
        run_table(entity)
        print(f'=======================================')

main()